In [1]:
# @title Submit Ed2 job
!pip install paramiko
!pip install PyDrive

from IPython.display import display, Javascript
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import ipywidgets as widgets
import json
from IPython.display import display
import getpass
import paramiko
import stat
import os
import time as timer
import subprocess
from urllib.parse import urlencode

# Define the file path
file_path = "clusters.json"

# Check if the file already exists
if not os.path.exists(file_path):
    # File does not exist, so create it
    with open(file_path, "w") as f:
        # Write some initial content if needed
        f.write('''
{
  "clusters": [
    {"hostname": "cc-login.campuscluster.illinois.edu", "batch_jobs": ["slurm"], "queues" : ["secondary", "IllinoisComputes", "IllinoisComputes-GPU"]},
    {"hostname": "stampede2.tacc.utexas.edu", "batch_jobs": ["slurm", "Kubernetes scheduler", "LSF"], "queues" : ["development", "normal","skx-normal", "skx-large", "icx-normal", "skx-dev"]},
    {"hostname": "docker", "batch_jobs":[], "queues":[]}
  ]
}
''')

with open('clusters.json', 'r') as json_file:
    clusters_data = json.load(json_file)
    cluster_options = [(cluster["hostname"], cluster["hostname"]) for cluster in clusters_data["clusters"]]

# Create a dictionary to store batch job options for each cluster
batch_jobs_dict = {cluster["hostname"]: cluster["batch_jobs"] for cluster in clusters_data["clusters"]}

# Create a dictionary to store batch job options for each cluster
queues_dict = {cluster["hostname"]: cluster["queues"] for cluster in clusters_data["clusters"]}

# HPC parameters
# Dropdown widget for cluster_input
cluster_input = widgets.Dropdown(options=cluster_options, description="Cluster:")

#cluster_input = widgets.Text(description="Cluster:")
username_input = widgets.Text(description="User name:")
user_password = widgets.Password(placeholder='Enter password',
    description='Password:',
    disabled=False
)
user_acc = widgets.Text(value="campusclusterusers", description="Account:")

# Dropdown widget for batch_job_input
batch_job_input = widgets.Dropdown(description="Batch job:")

# Dropdown widget for queue_input
queue_input = widgets.Combobox(
    placeholder='',
    options=[],
    ensure_option=True,  # True: restrict input to options; False: allow any input
    disabled=False
)
queue = widgets.HBox([widgets.Label(value="Queue name:"), queue_input])

# Set initial options for batch_job_input based on the first cluster
initial_cluster = cluster_options[0][0]
batch_job_input.options = batch_jobs_dict.get(initial_cluster, [])
queue_input.options = queues_dict.get(initial_cluster, [])

ed_binary_singularity_input = widgets.HBox([widgets.Label(value="Path to binary/singularity:"), widgets.Text(description="", value="${HOME}/ed2-intel.sif") ])
job_name_input = widgets.Text(value="ED2IN", description="Job name:")
num_nodes_input = widgets.IntText(value=1, description="No of nodes:")
runtime_input = widgets.Text(value="00:15:00", description="Runtime:")
work_folder_input = widgets.Text(placeholder="$HOME/ed/UUID", value="${HOME}/ed-demo", description="Work folder:")

# Callback function to handle changes in the cluster dropdown
def handle_cluster_change(change):
    selected_cluster = change.new

    # Disable or enable widgets based on the selected cluster
    if selected_cluster == "docker":
        batch_job_input.disabled = True
        username_input.disabled = True
        user_password.disabled = True
        user_acc.disabled = True
        queue_input.disabled = True
        ed_binary_singularity_input.children[1].disabled = True
        job_name_input.disabled = True
        num_nodes_input.disabled = True
        runtime_input.disabled = True
        work_folder_input.disabled = True
    else:
        batch_job_input.options = batch_jobs_dict.get(selected_cluster, [])
        queue_input.options = queues_dict.get(selected_cluster, [])
        batch_job_input.disabled = False
        username_input.disabled = False
        user_password.disabled = False
        user_acc.disabled = False
        queue_input.disabled = False
        ed_binary_singularity_input.children[1].disabled = False
        job_name_input.disabled = False
        num_nodes_input.disabled = False
        runtime_input.disabled = False
        work_folder_input.disabled = False

# Attach the callback function to the value attribute of the cluster dropdown
cluster_input.observe(handle_cluster_change, names='value')

# ED Path (Remote path)
ED2IN_path_input = widgets.Text(description="ED2IN path:", value="ED2IN-umbs.bg")
# met_folder_input = widgets.Text(description="MET folder:")
# glu_input = widgets.Text(description="GLU:")
# data_path_input = widgets.Text(description="Data path:")
header_file_input = widgets.Text(description="Header file path:", value="$HOME/ED-2.2_StartKit/ED2_InputData/SiteData/Santarem_Km83/MeteoDriver/Santarem_Km83_HEADER")

met_driver_input = widgets.Text(value="", description="MET driver:")
var_options = ['NL%FFILOUT', 'NL%SFILOUT', 'NL%GFILOUT', 'NL%SFILIN', 'NL%VEG_DATABASE', 'NL%SOIL_DATABASE', 'NL%LU_DATABASE', 'NL%THSUMS_DATABASE', 'NL%ED_MET_DRIVER_DB', 'NL%SOILSTATE_DB', 'NL%SOILDEPTH_DB']

# Function to handle addition of a new dropdown widget
def add_dropdown(button):
    # Remove selected options from var_options
    for dropdown in var_dropdowns.children:
        selected_option = dropdown.children[0].value
        if selected_option in var_options:
            var_options.remove(selected_option)

    # Add the new dropdown widget
    var_dropdowns.children += (create_dropdown(),)
    if len(var_options) == 1:
        add_button.disabled = True

# Function to create a new dropdown widget
def create_dropdown():
    return widgets.HBox([widgets.Dropdown(options=var_options, description='Replace:'), widgets.Text(placeholder="Enter the path")])

# Function to remove a dropdown widget
def remove_dropdown(button):
    if len(var_dropdowns.children) >= 1:
        var_dropdowns.children = var_dropdowns.children[:-1]
        var_option = var_dropdowns.children[-1].children[0].value
        if var_option not in var_options:
            var_options.append(var_option)
        add_button.disabled = False

# Create the "+" button
add_button = widgets.Button(description="Add more variable")
add_button.on_click(add_dropdown)

# Create the "-" button to remove dropdowns
remove_button = widgets.Button(description="Remove variable")
remove_button.on_click(remove_dropdown)

# Create the initial dropdown widget
initial_dropdown = create_dropdown()

# Create a container to hold all dropdowns
var_dropdowns = widgets.VBox([initial_dropdown])

# Display the widgets
#widgets.VBox([var_dropdowns, widgets.HBox([add_button, remove_button])])

# Create text box widget
text_box = widgets.Text(description='Selected Value:')

# ED simulation parameters Params
integrator_input = widgets.Text(description="Integrator:")

# ED model params
xml_input = widgets.Text(description="xml:")

# Submit button
submit_button = widgets.Button(description="Submit Job")

# Next notebook
next_button = widgets.Button(description="Next")

submitted_job_id = "10516422"

def redirect_to_notebook(event):
    print("Going to next notebook")
    target_url = "https://colab.research.google.com/drive/1NNSp6b4QahtFUOmnAroNDYVUkZEZDkB_?authuser=1"
    if submitted_job_id is not None:
      arguments = {
          "hostname": cluster_input.value,
          "username": username_input.value,
          "job_id": submitted_job_id
      }
      encoded_arguments = urlencode(arguments)
      if '?' in target_url:
        target_url += '&' + encoded_arguments
      else:
        target_url += '?' + encoded_arguments
    # Open the target notebook URL in a new browser tab
    display(Javascript(f'window.open("{target_url}", "_blank");'))


# Set the button's on_click event handler
next_button.on_click(redirect_to_notebook)


# Function to be executed on button click
def on_submit_button_click(button):
    hostname = cluster_input.value
    username = username_input.value
    password = user_password.value
    account = user_acc.value
    partition = queue_input.value
    job_name = job_name_input.value
    nodes = num_nodes_input.value
    time = runtime_input.value
    work_folder = work_folder_input.value
    path_ED2IN = ED2IN_path_input.value
    path_singularity_image = ed_binary_singularity_input.children[1].value

    # vars to be replaced in ED2IN
    header_file_path = header_file_input.value
    met_driver = met_driver_input.value
    vars = {}
    for dropdown in var_dropdowns.children:
        var_option = dropdown.children[0].value
        var_value = dropdown.children[1].value
        vars[var_option] = var_value

    # Batch job details
    ntasks_per_node = 16                    # Number of task (cores/ppn) per node
    output = "openmp_" + job_name + ".o%j"  # Name of batch job output file
    error = "openmp_" + job_name + ".e%j"   # Name of batch job error file
    mail_user = username + "@illinois.edu"        # Send email notifications
    mail_type = "BEGIN,END"                 # Type of email notifications to send

    ssh_client = paramiko.SSHClient()
    ssh_client.load_system_host_keys()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        ssh_client.connect(hostname, username=username, password=password, allow_agent=True)
    except:
        pass
    transport = ssh_client.get_transport()
    #transport.auth_password(username, getpass.getpass('Enter {0} Logon password :'.format(hostname)))
    sftp_client = paramiko.SFTPClient.from_transport(transport)

    #create the bat file
    with open(job_name + ".sbatch", 'w') as f:
        f.writelines("#!/bin/bash\n")
        f.writelines("#SBATCH --account=" + str(account) + "\n")
        f.writelines("#SBATCH --time=" + str(time) + "\n")
        f.writelines("#SBATCH --ntasks-per-node=" + str(ntasks_per_node) + "\n")
        f.writelines("#SBATCH --job-name=" + job_name + "\n")
        f.writelines("#SBATCH --partition=" + partition + "\n")
        f.writelines("#SBATCH --output=" + output + "\n")
        f.writelines("#SBATCH --error=" + error + "\n")
        f.writelines("#SBATCH --mail-user=" + mail_user + "\n")
        f.writelines("#SBATCH --mail-type=" + mail_type + "\n")
        f.writelines("\n")
        f.writelines("module load singularity" + "\n")
        if met_driver is not None and header_file_path is not None:
          f.writelines([f"sed -i /path_to/c{met_driver} {header_file_path}\n"])
        for key, value in vars.items():
          if key is not None and value is not None:
            abs_ED2IN_path = work_folder + path_ED2IN
            f.writelines([f"sed -i /{key}/c{value} {abs_ED2IN_path}\n"])
        f.writelines("singularity exec --bind " + work_folder + ":/data --no-home --pwd /data "
                       + path_singularity_image + " ed2 -f " + path_ED2IN)
    f.close()

    #transfer .bat file to cluster and run it
    sftp_client.put(job_name + ".sbatch", f"/home/{username}/" + job_name + ".sbatch")
    sftp_client.chmod(f"/home/{username}/" + job_name + ".sbatch", stat.S_IRWXU)
    _, stdo, stde = ssh_client.exec_command("sbatch " + job_name + ".sbatch")
    print(stde.read().decode())

    # Extract the job ID from the sbatch output
    result = stdo.read().decode()
    print(result)
    submitted_job_id = result.split()[3]

    sftp_client.close()
    ssh_client.close()
    transport.close()

# Attach the function to the button click event
submit_button.on_click(on_submit_button_click)

# Create Accordion widgets
hpc_accordion = widgets.Accordion(children=[widgets.VBox([cluster_input, username_input, user_password, user_acc, batch_job_input, queue, ed_binary_singularity_input, job_name_input, num_nodes_input, runtime_input, work_folder_input])])
#job_accordion = widgets.Accordion(children=[widgets.VBox([num_nodes_input, runtime_input, work_folder_input])])
ed_path_accordion = widgets.Accordion(children=[widgets.VBox([ED2IN_path_input, header_file_input])])
ed_vars_accordion = widgets.Accordion(children=[widgets.VBox([met_driver_input, var_dropdowns, widgets.HBox([add_button, remove_button])])])
ed_sim_accordion = widgets.Accordion(children=[widgets.VBox([integrator_input])])
ed_model_accordion = widgets.Accordion(children=[widgets.VBox([xml_input])])
submit_job_accordion = widgets.Accordion(children=[widgets.VBox([submit_button])])
next_job_accordion = widgets.Accordion(children=[widgets.VBox([next_button])])

# Set accordion titles
hpc_accordion.set_title(0, 'HPC Parameters')
#job_accordion.set_title(0, 'Job Parameters')
ed_path_accordion.set_title(0, 'ED Path Parameters')
ed_vars_accordion.set_title(0, 'ED Vars')
ed_sim_accordion.set_title(0, 'ED Simulation Params')
ed_model_accordion.set_title(0, 'ED Model Params')
submit_job_accordion.set_title(0, 'Submit Job')

# Display the accordions
display(hpc_accordion)
#display(job_accordion)
display(ed_path_accordion)
display(ed_vars_accordion)
#display(ed_sim_accordion)
#display(ed_model_accordion)
display(submit_job_accordion)
display(next_job_accordion)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 49.4 MB/s eta 0:00:00


Accordion(children=(VBox(children=(Dropdown(description='Cluster:', options=(('cc-login.campuscluster.illinois…

Accordion(children=(VBox(children=(Text(value='ED2IN-umbs.bg', description='ED2IN path:'), Text(value='$HOME/E…

Accordion(children=(VBox(children=(Text(value='', description='MET driver:'), VBox(children=(HBox(children=(Dr…

Accordion(children=(VBox(children=(Button(description='Submit Job', style=ButtonStyle()),)),), _titles={'0': '…

Accordion(children=(VBox(children=(Button(description='Next', style=ButtonStyle()),)),))

Going to next notebook


<IPython.core.display.Javascript object>